<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MISTRAL_devstral_2512_DEMO24DECV2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mistralai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.0/461.0 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.21.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.38.0 which is incompatible.
google-adk 1.21.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.38.0 which is incompatible.


In [2]:
import os
from mistralai import Mistral
from google.colab import userdata

# Initialize the client with your API key
api_key = userdata.get("MISTRAL_API_KEY")
client = Mistral(api_key=api_key)

# Fetch the list of models
models_response = client.models.list()

# Print the model IDs
for model in models_response.data:
    print(model.id)

mistral-medium-2505
mistral-medium-2508
mistral-medium-latest
mistral-medium
open-mistral-nemo
open-mistral-nemo-2407
mistral-tiny-2407
mistral-tiny-latest
mistral-large-2411
pixtral-large-2411
pixtral-large-latest
mistral-large-pixtral-2411
codestral-2508
codestral-latest
devstral-small-2507
devstral-medium-2507
devstral-2512
mistral-vibe-cli-latest
devstral-medium-latest
devstral-latest
labs-devstral-small-2512
devstral-small-latest
mistral-small-2506
mistral-small-latest
labs-mistral-small-creative
magistral-medium-2509
magistral-medium-latest
magistral-small-2509
magistral-small-latest
voxtral-mini-2507
voxtral-mini-latest
voxtral-small-2507
voxtral-small-latest
mistral-large-2512
mistral-large-latest
ministral-3b-2512
ministral-3b-latest
ministral-8b-2512
ministral-8b-latest
ministral-14b-2512
ministral-14b-latest
open-mistral-7b
mistral-tiny
mistral-tiny-2312
pixtral-12b-2409
pixtral-12b
pixtral-12b-latest
ministral-3b-2410
ministral-8b-2410
codestral-2501
codestral-2412
codestra

In [4]:
import json
import io
import sys
import traceback
from mistralai import Mistral
from google.colab import userdata

# 1. Setup
try:
    api_key = userdata.get("MISTRAL_API_KEY")
    client = Mistral(api_key=api_key)
    MODEL = "devstral-2512"
except Exception as e:
    print(f"❌ Setup Error: {e}")
    sys.exit(1)

# 2. Tool Logic
# We use a dictionary to simulate a real file system for the agent to interact with
virtual_fs = {"main.py": "", "database.py": "# DB Logic", "requirements.txt": "mistralai"}

def list_files():
    return json.dumps({"files": list(virtual_fs.keys())})

def write_to_file(filename: str, content: str):
    virtual_fs[filename] = content
    return json.dumps({"status": "success", "message": f"Updated {filename}"})

def execute_python_code(code: str):
    output = io.StringIO()
    original_stdout = sys.stdout
    sys.stdout = output
    try:
        # We provide a clean global dict for every execution
        exec_globals = {}
        exec(code, exec_globals)
        captured_output = output.getvalue().strip()
        return json.dumps({
            "status": "success",
            "stdout": captured_output or "Executed (no stdout)."
        })
    except Exception:
        return json.dumps({"status": "error", "message": traceback.format_exc()})
    finally:
        sys.stdout = original_stdout

# 3. Tool Definitions
tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Lists workspace files.",
            "parameters": {"type": "object", "properties": {}}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "write_to_file",
            "description": "Writes or overwrites a file in the workspace.",
            "parameters": {
                "type": "object",
                "properties": {
                    "filename": {"type": "string"},
                    "content": {"type": "string"}
                },
                "required": ["filename", "content"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "execute_python_code",
            "description": "Runs Python code and returns output.",
            "parameters": {
                "type": "object",
                "properties": {"code": {"type": "string"}},
                "required": ["code"]
            }
        }
    }
]

# 4. The Agentic Loop
messages = [
    {
        "role": "system",
        "content": "You are a Devstral coding agent. Always check the file list first. If you refactor code, write it to 'main.py' before finishing."
    },
    {"role": "user", "content": "Refactor Fibonacci with memoization, test n=10, and save it to the workspace."}
]

print(f"🚀 STARTING AGENT ({MODEL})\n" + "-"*50)

for i in range(7): # Increased limit to allow for write steps
    response = client.chat.complete(model=MODEL, messages=messages, tools=tools)
    assistant_msg = response.choices[0].message
    messages.append(assistant_msg)

    if assistant_msg.content:
        print(f"🤖 THOUGHT: {assistant_msg.content}")

    if not assistant_msg.tool_calls:
        print("\n" + "="*50 + "\n✅ TASK COMPLETE\n" + "="*50)
        break

    for tool_call in assistant_msg.tool_calls:
        name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)

        print(f"🛠️  ACTION: {name}")

        if name == "list_files":
            result = list_files()
        elif name == "write_to_file":
            result = write_to_file(args['filename'], args['content'])
            print(f"💾 Saved to {args['filename']}")
        else:
            print(f"📝 RUNNING CODE...")
            result = execute_python_code(args['code'])

        print(f"📥 RESULT: {result}\n")

        messages.append({
            "role": "tool",
            "name": name,
            "tool_call_id": tool_call.id,
            "content": result
        })

# Show the final state of the virtual filesystem
print("\n📂 FINAL WORKSPACE CONTENT (main.py):")
print(virtual_fs["main.py"])

🚀 STARTING AGENT (devstral-2512)
--------------------------------------------------
🛠️  ACTION: list_files
📥 RESULT: {"files": ["main.py", "database.py", "requirements.txt"]}

🛠️  ACTION: write_to_file
💾 Saved to main.py
📥 RESULT: {"status": "success", "message": "Updated main.py"}

🛠️  ACTION: execute_python_code
📝 RUNNING CODE...
📥 RESULT: {"status": "error", "message": "Traceback (most recent call last):\n  File \"/tmp/ipython-input-2289606160.py\", line 35, in execute_python_code\n    exec(code, exec_globals)\n  File \"<string>\", line 1, in <module>\nModuleNotFoundError: No module named 'main'\n"}

🛠️  ACTION: execute_python_code
📝 RUNNING CODE...
📥 RESULT: {"status": "success", "stdout": "Fibonacci(10) = 55"}

🤖 THOUGHT: The Fibonacci function with memoization has been refactored, tested with `n=10`, and saved to the workspace. The result for `Fibonacci(10)` is `55`.

✅ TASK COMPLETE

📂 FINAL WORKSPACE CONTENT (main.py):
def fibonacci(n, memo={}):
    if n in memo:
        return